# Datasets, Dataloaders and Main model objects

### Import Libraries

In [1]:
from __future__ import print_function
import numpy as np
import string
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import time
import pickle
import random
import math
from torch.utils.data import Dataset
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader, Subset
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

In [2]:
!pip install nbformat

## Initialization

In [3]:
# 
seed = 230729
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
max_code = pickle.load(open('events_maxcode.p', 'rb')) + 1
assert max_code==127, "EVENTS MAX CODE changed?"

We know that not all patients have the same number of visit dates, therefore, we need to find what is the maximum number of visit dates for any given patient

In [4]:
patients_max_visits = 505

In preparation to run the models training on CUDA, we need to make sure that we do have a device and load the tensors and model to CUDA

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce GTX 1060 6GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


## Loading data from files

These files were generated from SQL queries contained in notes_extraction.ipynb and events_extraction.ipynb notebooks.

In [6]:
def load_events_dataset_object(prefix=''):
    return pickle.load( open(prefix + "events_item.p", "rb" )), pickle.load(open(prefix + "events_value.p", "rb"))

In [7]:
def load_notes_dataset_object(prefix = ''):
    
    patient_subject_id = np.load(prefix + 'subject_id.npy', allow_pickle=True).tolist()
    patients_notes_fetures = np.load(prefix + 'patients_notes_fetures.npy', allow_pickle=True)
    index_0 = np.load(prefix + 'index_0.npy', allow_pickle=True)
    index_1 = np.load(prefix + 'index_1.npy', allow_pickle=True)
    patient_mortality = np.load(prefix + 'patient_mortality.npy', allow_pickle=True)

    return patient_subject_id, patients_notes_fetures, index_0, index_1, patient_mortality

## Dataset definition

In [8]:
class NotesEventsDataset(Dataset):
    
    def __init__(self, patient_id, patients_notes, notes_mask, events_items, events_values, mortality):
        
        self.patient_id = patient_id
        len_events_patients = len(events_items['subject_id'].unique())
        self.x = patients_notes.to(device, non_blocking=True)
        self.notes_mask = notes_mask.to(device, non_blocking=True)
        self.y = mortality.to(device, non_blocking=True)
        self.items = events_items.groupby('subject_id').agg('codes').apply(list).values
        self.values = events_values.groupby('subject_id').agg('values').apply(list).values
        assert len(self.x) == len_events_patients, 'Notes patients and events patients counts do not match!'
        r = random.randrange(len(self.x))
        assert events_items['subject_id'].unique()[r] == self.patient_id[r], 'Notes and events patient id=' + str(r) + ' does not match'
    
    def __len__(self):

        return len(self.x)
    
    def __getitem__(self, index):
        
        events = np.zeros([len(self.items[index]), max_code])

        for i, codes in enumerate(self.items[index]):
            for j, code in enumerate(codes):
                v = self.values[index][i][j]
                events[i, code] = v if not math.isnan(v) else 0.0
        

        return(self.x[index].to_dense(), self.notes_mask[index].to_dense(), events, self.y[index])

In [1]:
def create_dataset (cohort_type = 'original'):
    """
    cohort_type = 'original' -> Unbalanced cohort will be created
    cohort_type = 'essential'  ->  Unbalanced cohort with just the minimum set of events features
    cohort_type = 'balanced_train' -> Balanced cohort for training will be created
    cohort_type = 'balanced_test' -> Balanced cohort for testing will be created
    """
    notes_prefix = "orig_" if cohort_type in ['original','essential'] else "train_" if cohort_type == 'balanced_train' else "test_"
    subject_id, patients_notes_fetures, index_0, index_1, patient_mortality= load_notes_dataset_object(prefix = notes_prefix)
    index = [index_0, index_1]
    patients_notes_fetures = torch.sparse_coo_tensor(index, patients_notes_fetures, (len(subject_id),patients_max_visits,200), dtype = torch.float)
    
    ones = np.ones((len(index_0),200))
    notes_mask = torch.sparse_coo_tensor(index, ones, (len(subject_id),patients_max_visits,200), dtype = torch.float)
    
    patient_mortality = torch.from_numpy(patient_mortality).float()    
    
    events_prefix = "" if cohort_type=='original' else cohort_type +"_" 
    events_items, events_values = load_events_dataset_object(events_prefix)

    assert len(events_items)==len(events_values) and len(events_values['subject_id'].unique()) == len(events_items['subject_id'].unique()) == len(patient_mortality) , "Wrong events dataframes?"
        
    dataset = NotesEventsDataset(subject_id, patients_notes_fetures, notes_mask, events_items, events_values, patient_mortality)
    assert len(patient_mortality) == len(dataset), 'Wrong dataset length!'
    print ("Number of Patients:", len(patient_mortality))


    return dataset

## DataLoaders definition

### Batch size

In [10]:
batch_size = 50

In [11]:
def collate_fn(data):
    x, notes_mask, events, mortality_flag = zip(*data)
    
    maxvisits = max([len(p) for p in events])
    
    events_result = torch.tensor([np.concatenate((p, np.zeros([maxvisits - len(p), max_code]))) for p in events]).float()
    events_mask = torch.tensor([np.concatenate((np.ones(len(p)), np.zeros(maxvisits - len(p)))) for p in events]).int()
    x = torch.stack(x)
    notes_mask = torch.stack(notes_mask)
    mortality_flag = torch.stack(mortality_flag)
    events_result = events_result.to(device, non_blocking=True)
    events_mask = events_mask.to(device, non_blocking=True)
    return x, notes_mask, events_result, events_mask, mortality_flag

Now we create the data loaders, splitting the dataset on 80% train, 20% validation.

### Dataloader for the unbalanced (original) cohort.

In [12]:
def get_unbalanced_dataloaders (max_size = 0):

    dataset = create_dataset('original')
    if (max_size > 0):
        print ("***** Slicing to " + str(max_size))
        dataset = Subset(dataset, np.arange(max_size))

    split = int(len(dataset)*0.8)
    lengths = [split, len(dataset) - split]

    train_dataset, val_dataset = random_split(dataset, lengths)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

    return train_loader, val_loader

### Dataloader for the unbalanced cohort but with the reduced (essential) features set.

In [13]:
def get_essential_dataloaders (max_size = 0):

    dataset = create_dataset('essential')
    if (max_size > 0):
        print ("***** Slicing to " + str(max_size))
        dataset = Subset(dataset, np.arange(max_size))

    split = int(len(dataset)*0.8)
    lengths = [split, len(dataset) - split]

    train_dataset, val_dataset = random_split(dataset, lengths)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

    return train_loader, val_loader

### Dataloader for the balanced cohort (data up-sampling) for "dead" patients to eliminate class imbalance).

In [2]:
def get_balanced_dataloaders (max_size = 0):

    print ("* Train dataset *")
    balanced_train_dataset = create_dataset('balanced_train')
    print ("* Test dataset *")
    balanced_test_dataset = create_dataset('balanced_test')

    if (max_size > 0):
        print ("***** Slicing to " + str(max_size))
        balanced_train_dataset = Subset(balanced_train_dataset, np.arange(max_size))
        balanced_test_dataset = Subset(balanced_test_dataset, np.arange(max_size))

    train_loader = DataLoader(balanced_train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
    val_loader = DataLoader(balanced_test_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

    return train_loader, val_loader

## Model Definition

### Notes Network

In [14]:
%run notes_module.ipynb

### Events Network

In [15]:
%run events_module.ipynb

### Final Classifier Network

In [16]:
class OutNet(nn.Module):
    def __init__(self, notes_embeddings=128, events_embeddings=128):
        super().__init__()
        
        self.notes = NotesRNN(notes_embeddings)
        self.events = EventsRNN(max_code, emb_size=events_embeddings)
        if torch.cuda.device_count() >0:
            self.notes.cuda()
            self.events.cuda()
        self.fc1 = nn.Linear(notes_embeddings + events_embeddings, 128)
        self.fc2 = nn.Linear(128, 32)
        self.dropout = nn.Dropout()
        self.fc3 = nn.Linear(32, 1)
        self.sig = nn.Sigmoid()    
    
    def forward(self, x, masks, events, events_masks):
        
        
        events_emb = self.events(events, events_masks)
        
        notes_emb = self.notes(x, masks)
        #notes_emb = torch.rand(events_emb.shape[0],128)
        #events_emb = torch.rand(notes_emb.shape[0], 128)
        
        
        joint_emb = torch.cat([notes_emb, events_emb], dim=1)
        
        fc1 = self.fc1(joint_emb)
        fc2 = self.dropout(self.fc2(fc1))
        fc3 = self.fc3(fc2)
        output = self.sig(fc3).flatten()
        return output

#outNet = OutNet()
#outNet

In [17]:
def create_model_and_optimizer():
    model = OutNet(notes_embeddings=128, events_embeddings=128)
    if torch.cuda.device_count() >0:
        model.cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9, nesterov = True)
    return model, optimizer

## Model training and evaluation

In [18]:
def train(model, train_loader, n_epochs):
    model.train() # prep model for training
    
    for epoch in range(n_epochs):
        curr_epoch_loss = []
        print('Batch :', end = ' ')
        for step, batch in enumerate(train_loader):
            if step % 10 == 0 and step>0:
                print(str(step)+',', end=' ' )
            x, masks, events, events_masks, labels = batch
        
            """ Step 1. clear gradients """
            optimizer.zero_grad()
            """ Step 2. evaluate model ouput  """
            probs = model(x, masks, events, events_masks)
            """ Step 3. Calculate loss  """
            loss = criterion(probs, labels)
            """ Step 4. Backward propagation  """
            loss.backward()
            """ Step 5. optimization """
            optimizer.step()
            """ Step 6. record loss """
            curr_epoch_loss.append(loss.cpu().data.numpy())
        
        
        print(f"Epoch {epoch}: curr_epoch_loss={np.mean(curr_epoch_loss)}")
    return model

In [19]:
def eval_model(model, val_loader):
    model.eval()
    val_labels = []
    val_probs = []
    
    for step, batch in enumerate(val_loader):
        x, masks, events, events_masks, labels = batch
        
        with torch.no_grad():
            
            probs = model(x, masks, events, events_masks)
            val_labels.extend(labels.detach().cpu().numpy().tolist())
            val_probs.extend(probs.detach().cpu().numpy().reshape(-1).tolist())

    precision, recall, f1, _ = precision_recall_fscore_support(val_labels, np.array(val_probs)>0.5, average='binary')
    roc_auc = roc_auc_score(val_labels, val_probs)
    
    return precision, recall, f1, roc_auc

In [20]:
def train_and_eval(model, train_loader, val_loader, n_epochs=10, filename='model.pt'):
    t0 = time.time()
    train(model, train_loader, n_epochs)
    t1 = time.time()
    processing_time = t1-t0
    print('Model Training time: ' + str(processing_time))
    
    p, r, f, roc_auc = eval_model(model, val_loader)
    print ("Learning rate: " + str(learning_rate))
    print("Model Training time: " + str(processing_time))
    print("Precision = ",p)
    print("Recall    = ", r)
    print("F1        = ", f)
    print("ROC AUC   = ", roc_auc)
    print(p,"\t",r,"\t",f,"\t",roc_auc)
    
    if filename is not None:
        torch.save(model.state_dict(), filename)
    return p, r, f, roc_auc

In [21]:
def load_and_eval(model, modelfilename, val_loader):  
    print ("**LOADING MODEL** from " + modelfilename)
    model.load_state_dict(torch.load(modelfilename, map_location=torch.device('cpu')))
    p, r, f, roc_auc = eval_model(model, val_loader)
    print ("")
    print("Precision = ",p)
    print("Recall    = ", r)
    print("F1        = ", f)
    print("ROC AUC   = ", roc_auc)
    print(p,"\t",r,"\t",f,"\t",roc_auc)